In [6]:
import pandas as pd
import numpy as np


# Load data from CSV in 'data/' folder
data_path = r"D:\Conestoga college\Ecommerce-data-engineering-lab-kapil\data\ecommerce_data.csv"
df = pd.read_csv(data_path)

# Show first 3 rows
df.head(3)


,Order_Date,Time,Aging,Customer_Id,Gender,Device_Type,Customer_Login_type,Product_Category,Product,Sales,Quantity,Discount,Profit,Shipping_Cost,Order_Priority,Payment_method,shipping_city
0,2018-01-02,10:56:33,8.0,37077,Female,Web,Member,Auto & Accessories,Car Media Players,140.0,1.0,0.3,46.0,4.6,Medium,credit_card,Montreal
1,2018-07-24,20:41:37,2.0,59173,Female,Web,Member,Auto & Accessories,Car Speakers,211.0,1.0,0.3,112.0,11.2,Medium,credit_card,Calgary
2,2018-11-08,08:38:49,8.0,41066,Female,Web,Member,Auto & Accessories,Car Body Covers,117.0,5.0,0.1,31.2,3.1,Critical,credit_card,Vancouver


In [7]:
df = pd.read_csv(data_path)
df['shipping_city'] = np.random.choice(['Toronto', 'Vancouver', 'Calgary', 'Montreal'], size=len(df))
df.to_csv(data_path, index=False)  # Overwrite with new column


import pandas as pd

# Load data from CSV in 'data/' folder
data_path = "data/ecommerce_data.csv"
df = pd.read_csv(data_path)

# Show first 3 rows
df.head(3)


Step 3: Transaction Class and OO Data Structure

In [8]:
from datetime import datetime
from typing import Optional

class Transaction:
    def __init__(self, date: str, customer_id: str, product: str, price: float,
                 quantity: int, coupon_code: Optional[str], shipping_city: str):
        self.date = date
        self.customer_id = customer_id
        self.product = product
        self.price = price
        self.quantity = quantity
        self.coupon_code = coupon_code
        self.shipping_city = shipping_city
        
    def clean(self):
        # Convert date to datetime object
        try:
            self.date = datetime.strptime(self.date, "%Y-%m-%d")
        except Exception:
            self.date = None
        
        # Clean price and quantity
        if self.price < 0:
            self.price = abs(self.price)
        if self.quantity < 0:
            self.quantity = abs(self.quantity)
        
        # Normalize coupon_code, convert empty strings to None
        if self.coupon_code == "" or self.coupon_code is None:
            self.coupon_code = None
        
        # Standardize city names (simple example: strip whitespace)
        self.shipping_city = self.shipping_city.strip().title()
        
    def total_price(self) -> float:
        return self.price * self.quantity


Step 4: Bulk Loader

In [9]:
from typing import List

def load_transactions(csv_path: str) -> List[Transaction]:
    df = pd.read_csv(csv_path)
    transactions = []
    
    for _, row in df.iterrows():
        try:
            txn = Transaction(
                date=row['Order_Date'],
                customer_id=str(row['Customer_Id']),
                product=row['Product'],
                price=float(row['Sales']),
                quantity=int(float(row['Quantity'])),  # Handles "3.0" or "3"
                coupon_code=str(row['Discount']) if pd.notna(row['Discount']) else None,
                shipping_city="City Placeholder"  # Replace if you have a real city field
            )
            transactions.append(txn)
        except (ValueError, TypeError, KeyError) as e:
            print(f"Skipping row due to error: {e}")
            continue
    
    return transactions

transactions = load_transactions(data_path)


Skipping row due to error: cannot convert float NaN to integer
Skipping row due to error: cannot convert float NaN to integer


Step 5: Quick Profiling

In [10]:
prices = [txn.price for txn in transactions]
cities = {txn.shipping_city for txn in transactions}

print(f"Price stats - Min: {min(prices):.2f}, Mean: {sum(prices)/len(prices):.2f}, Max: {max(prices):.2f}")
print(f"Unique shipping cities: {len(cities)}")


Price stats - Min: 33.00, Mean: nan, Max: 250.00
Unique shipping cities: 1


Identified dirty data cases:

1. Negative or zero prices and quantities.
2. Missing or malformed dates.
3. Empty or inconsistent coupon codes.
4. Inconsistent city name formatting (e.g., " new york", "New York ").


In [11]:
# Before cleaning count
dirty_count = sum(1 for txn in transactions if txn.price <= 0 or txn.quantity <= 0 or txn.date is None)
print(f"Dirty records before cleaning: {dirty_count}")

# Clean all transactions
for txn in transactions:
    txn.clean()

# After cleaning count
dirty_count_after = sum(1 for txn in transactions if txn.price <= 0 or txn.quantity <= 0 or txn.date is None)
print(f"Dirty records after cleaning: {dirty_count_after}")


Dirty records before cleaning: 0
Dirty records after cleaning: 0


In [12]:
def parse_coupon(coupon_code: Optional[str]) -> float:
    # Example: coupon code "SAVE10" means 10% discount
    if coupon_code and coupon_code.startswith("SAVE"):
        try:
            return float(coupon_code[4:]) / 100
        except:
            return 0.0
    return 0.0

for txn in transactions:
    txn.discount = parse_coupon(txn.coupon_code)


In [13]:
from datetime import datetime

reference_date = datetime.now()

for txn in transactions:
    if txn.date:
        txn.days_since_purchase = (reference_date - txn.date).days
    else:
        txn.days_since_purchase = None


In [15]:
from collections import defaultdict
import math

revenue_per_city = defaultdict(float)

for txn in transactions:
    city = txn.shipping_city
    # Check if city is not None or nan
    if city and not (isinstance(city, float) and math.isnan(city)):
        total = txn.total_price() * (1 - txn.discount)
        revenue_per_city[city] += total

# Print the revenue per city
for city, revenue in revenue_per_city.items():
    print(f"{city}: ${revenue:.2f}")


City Placeholder: $nan


In [ ]:
import json
import pyarrow.parquet as pq
import pyarrow as pa

# Save cleaned data to JSON
json_path = "data/cleaned_transactions.json"
with open(json_path, "w") as f:
    json.dump([txn.__dict__ for txn in transactions], f, default=str)

# Save cleaned data to Parquet using pandas intermediary
df_cleaned = pd.DataFrame([txn.__dict__ for txn in transactions])
parquet_path = "data/cleaned_transactions.parquet"
df_cleaned.to_parquet(parquet_path)


Using Object-Oriented Programming (OOP) simplified the data cleaning and transformation processes by encapsulating related data and behaviors in the `Transaction` class. This structure enhanced code readability and reusability, allowing individual transaction instances to clean and compute values independently. It also made bulk operations and debugging easier by isolating functionality into methods, improving maintainability and scalability.
